In [1]:
import pandas as pd
import re
import numpy as np
from fastai.tabular import *
from collections import Counter

In [2]:
dtypes = {
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [3]:
df_tr = pd.read_csv('train.csv',dtype = dtypes)
df_tr.fillna(-1,inplace=True)

/home/dlo/anaconda3/envs/fastai_gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_te = pd.read_csv('test.csv',dtype = dtypes)
df_te.fillna(-1,inplace=True)

### Notes from feat exploration:

## V2 - let's be greedy and encode every category without cardinality limit.
## Also when we float encode a column, let's not drop the categorical version

- [X] FillNaN by -1
- [X] MachineIdentifier' : drop 
- [X] AvSigVersion: get date from the signature and add a few features from the date
- [X] AvSigVersion: Split on dots, keep only col 1 and 2, replace weird symbol in col 1 by 273, to float, normalize
- [X] AvSigVersion: Get index for validation set that looks like test set
- [X]  AVProductStatesIdentifier : enc_col_nosizelimit 
- [X]  CityIdentifier: enc_col_nosizelimit 
- [X]  Census_OEMNameIdentifier : enc_col_nosizelimit 
- [X]  Census_ProcessorModelIdentifier : enc_col_nosizelimit 
- [X]  Census_PrimaryDiskTotalCapacity: int(log2 + 1.5) 
- [X]  Census_SystemVolumeTotalCapacity: int(log2 + 1.5) 
- [X]  Census_TotalPhysicalRAM: int(log2 + 1.5) 
- [X]  Census_InternalBatteryNumberOfCharges: int(log2 + 1.5) 
- [X]  Census_FirmwareVersionIdentifier : one version with groupby(FirmwareManufacturer).normalize if we find out how to normalize in the test set with train stats, otherwise just normalize the stuff without group 
- [X] Census_FirmwareVersionIdentifier :one version with enc_col_nosizelimit, 
- [X]  EngineVersion: split, keep col 1,2, normalize 
- [X]  AppVersion: split, cat encode col 1, float normalize col 2,3 
- [X]  OsVer: split, cat encode 
- [X]  As all the rest of the data is treated as categorical data, let's also pass it through enc_col_nosizelimit  



Let's start with special treatments, splitting normalizing, then loop over the rest with enc_col or enc_col_nosizelimit <br>

In [5]:
## HELPER FUNCTIONS

def get_col_enc_dict(c,limit) :
    """
    Takes as input c, a pd.Series.
    Counts uniques values in c
    Creates a list of values that have at least 10 occurences in c.
    The first element in the list is the most frequent element in c, etc.
    Keeps the first 500 elements, store it in most_comm_vals
    Returns a dict mapping the most 500 frequent values to their position (+1)
    in the list. To be used to int encode c.
    """
    ct = Counter(c)
    most_comm_vals = [k for k,v in ct.most_common() if ((v > 50) & (k != -1))]
    """if limit == False :
        most_comm_vals = [k for k,v in ct.most_common() if ((v > 100) & (k != -1))]
    else :
        most_comm_vals = [k for k,v in ct.most_common() if ((v > 30) & (k != -1))][:500]"""
    most_comm_dict = {k : v+1 for v,k in enumerate(most_comm_vals)}
    most_comm_dict[-1] = -1
    return most_comm_dict

def enc_col(col,limit,d = None) :
    """
    Elements of that c are in d are mapped to their values in d
    Elements not in c and are not NaNs are mapped to 0
    NaNs were mapped to -1 at the beginning of the notebook and
    are kept to -1
    """
    c = col.copy()
    if d is None : d = get_col_enc_dict(c,limit)
    in_keys = c.isin(d.keys())
    c.loc[in_keys] = c.map(d)
    c.loc[~in_keys] = 0
    
    return c,d

def enc_tr_te_col(tr,te,limit) :
    tr, d = enc_col(tr,limit)
    te, _ = enc_col(te,limit,d)
    tr = tr.astype('category').cat.as_ordered()
    te = pd.Categorical(te, categories = tr.cat.categories, ordered = True)
    te = pd.Series(te)
    return tr, te


splitter = lambda x : x.split(".")

def log_to_int(s) : return (100*np.log2(s + 1.5)).astype('int16')

In [6]:
real_cols = []

In [7]:
# drop
col = 'MachineIdentifier'

df_tr.drop(columns = col,inplace=True)
df_te.drop(columns = col,inplace=True)

In [8]:
"""
AvSigVersion : add a timestamp from the avsig version
add columns based on the timestamp (day, month, ..)
process buggy rows added by buggy avsig
"""

col = 'AvSigVersion'

weird_symbol = "2&#x17;3"

df_tr[col] = df_tr[col].apply(lambda x : x.replace(weird_symbol,'273'))
datedict = np.load('AvSigVersionTimestamps.npy') # you can get than from a kaggle kernel
datedict = datedict[()]

df_tr['Date'] = df_tr[col].map(datedict) # Drops the field Date
add_datepart(df_tr,'Date')

df_te['Date'] = df_te[col].map(datedict) # Drops the field Date
add_datepart(df_te,'Date')

"""
New columns added from the date by add_datepart():
['Year', 'Month', 'Week','Day', 'Dayofweek',
'Dayofyear', 'Is_month_end', 'Is_month_start',
'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
'Is_year_start','Elapsed']
Some AvSigVersion are equal to 0.0.0.0 which gives an unknown
date, and erroneous added columns. Need to handle them
"""

err_tr = df_tr['Year'].isnull()
err_te = df_te['Year'].isnull()

cols_ = ['Is_month_end','Is_month_start',
'Is_quarter_end','Is_quarter_start',
'Is_year_end','Is_year_start',
'Year', 'Month', 'Week','Day', 'Dayofweek',
'Dayofyear'] 
# These are categorical variables
df_tr.loc[err_tr,cols_] = -1
df_te.loc[err_te,cols_] = -1

col_ = 'Elapsed'
# This is a unix timestamp64ns or something, test set has nearly
# only values not seen in the train set, turning it into real number
# and normalizing.

trm = df_tr.loc[~err_tr,col_].min()
trM = df_tr.loc[~err_tr,col_].max()

# Setting erroneous cols to mid range of non-erroneous cols
df_tr.loc[err_tr,col_] = (trM + trm)/2
df_te.loc[err_te,col_] = (trM + trm)/2

# Normalizing
df_tr[col_] = (df_tr[col_] - trm)/(trM - trm)
df_te[col_] = (df_te[col_] - trm)/(trM - trm)

real_cols.append('Elapsed')

In [9]:
"""
AvSigVersion : split on dots, keep only col 1 and 2, to float, normalize.
"""
col = 'AvSigVersion'

tr = pd.DataFrame([splitter(o) for o in df_tr[col].values])
#df_tr.drop(columns = col, inplace=True)

tr = tr.loc[:,[1,2]]
tr = tr.astype('float32')
df_tr['valid_idx'] = (tr[1] >= 275).astype(int) # Storing index for validation set that looks like test set
# tr[1] has a kind of regular range between 155 and 280, but a hole between 0 and 155
tr.loc[tr[1] == 0,1] = 210 
trm = tr.min()
trM = tr.max()
tr = (tr - trm)/(trM-trm)

tr.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_tr = pd.concat([df_tr,tr],axis=1)

te = pd.DataFrame([splitter(o) for o in df_te[col].values])
#df_te.drop(columns = col, inplace=True)

te = te.loc[:,[1,2]]
te = te.astype('float32')
# te[1] has a kind of regular range between 155 and 280, but a hole between 0 and 155
te.loc[te[1] == 0,1] = 210 
te = (te - trm)/(trM-trm)

te.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_te = pd.concat([df_te,te],axis=1)

real_cols.extend([f'{col}.norm.0',f'{col}.norm.1'])

In [10]:
# We want both a normalized and a categorical version, let's do the normalized

col = 'Census_FirmwareVersionIdentifier'

trm = df_tr[col].min()
trM = df_tr[col].max()
df_tr[f"{col}.norm"] = (df_tr[col] - trm)/(trM-trm)
df_te[f"{col}.norm"] = (df_te[col] - trm)/(trM-trm)

real_cols.append(f"{col}.norm")

In [11]:
col = 'EngineVersion'

tr = pd.DataFrame([splitter(o) for o in df_tr[col].values])
#df_tr.drop(columns = col, inplace=True)

tr = tr.loc[:,[2,3]]
tr = tr.astype('float32')
trm = tr.min()
trM = tr.max()
tr = (tr - trm)/(trM-trm)

tr.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_tr = pd.concat([df_tr,tr],axis=1)

te = pd.DataFrame([splitter(o) for o in df_te[col].values])
#df_te.drop(columns = col, inplace=True)

te = te.loc[:,[2,3]]
te = te.astype('float32')
te = (te - trm)/(trM-trm)

te.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_te = pd.concat([df_te,te],axis=1)

real_cols.extend([f'{col}.norm.0',f'{col}.norm.1'])

In [12]:
col = 'AppVersion'

tr = pd.DataFrame([splitter(o) for o in df_tr[col].values])
#df_tr.drop(columns = col, inplace=True)

tr1 = tr.loc[:,[1]] # Cat
tr1.columns = [f"{col}.cat"]

tr23 = tr.loc[:,[2,3]] # Real - normalizing
tr23 = tr23.astype('float32')
trm = tr23.min()
trM = tr23.max()
tr23 = (tr23 - trm)/(trM-trm)
tr23.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_tr = pd.concat([df_tr,tr1,tr23],axis=1)

te = pd.DataFrame([splitter(o) for o in df_te[col].values])
#df_te.drop(columns = col, inplace=True)

te1 = te.loc[:,[1]] # Cat
te1.columns = [f"{col}.cat"]

te23 = te.loc[:,[2,3]] # Real 
te23 = te23.astype('float32')
te23 = (te23 - trm)/(trM-trm)

te23.columns = [f'{col}.norm.0',f'{col}.norm.1']
df_te = pd.concat([df_te,te1,te23],axis=1)

real_cols.extend([f'{col}.norm.0',f'{col}.norm.1'])

In [13]:
col = 'OsVer'

tr = pd.DataFrame([splitter(o) for o in df_tr[col].values])
df_tr.drop(columns = col, inplace=True)
tr.columns = [f"{col}.cat.{i}" for i in range(4)]
df_tr = pd.concat([df_tr,tr],axis = 1)

te = pd.DataFrame([splitter(o) for o in df_te[col].values])
df_te.drop(columns = col, inplace=True)
te.columns = [f"{col}.cat.{i}" for i in range(4)]
df_te = pd.concat([df_te,te],axis = 1)

In [14]:
to_log = ['Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity',
         'Census_TotalPhysicalRAM','Census_InternalBatteryNumberOfCharges']

for c in to_log :
    
    df_tr[c] = log_to_int(df_tr[c])
    df_te[c] = log_to_int(df_te[c])
    print(f"{c} done")

Census_PrimaryDiskTotalCapacity done
Census_SystemVolumeTotalCapacity done
Census_TotalPhysicalRAM done
Census_InternalBatteryNumberOfCharges done


In [14]:
to_enc_with_size_limit = ['AVProductStatesIdentifier','Census_OEMNameIdentifier']

for c in to_enc_with_size_limit :
    df_tr[c],df_te[c] = enc_tr_te_col(df_tr[c],df_te[c],limit = True)
    print(f"{c} done")
    

AVProductStatesIdentifier done
Census_OEMNameIdentifier done


In [16]:
to_enc

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'IsBeta',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'DefaultBrowsersIdentifier',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'HasTpm',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'AutoSampleOptIn',
 'PuaMode',
 'SMode',
 'IeVerIdentifier',
 'SmartScreen',
 'Firewall',
 'UacLuaenable',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_ProcessorClass',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Cen

In [17]:
to_enc = [c for c in df_te.columns if (c not in real_cols) & (c != "valid_idx")]

for c in to_enc :
    df_tr[c],df_te[c] = enc_tr_te_col(df_tr[c],df_te[c],limit = False)
    print(f"{c} done")

ProductName done
EngineVersion done
AppVersion done
AvSigVersion done
IsBeta done
RtpStateBitfield done
IsSxsPassiveMode done
DefaultBrowsersIdentifier done
AVProductStatesIdentifier done
AVProductsInstalled done
AVProductsEnabled done
HasTpm done
CountryIdentifier done
CityIdentifier done
OrganizationIdentifier done
GeoNameIdentifier done
LocaleEnglishNameIdentifier done
Platform done
Processor done
OsBuild done
OsSuite done
OsPlatformSubRelease done
OsBuildLab done
SkuEdition done
IsProtected done
AutoSampleOptIn done
PuaMode done
SMode done
IeVerIdentifier done
SmartScreen done
Firewall done
UacLuaenable done
Census_MDC2FormFactor done
Census_DeviceFamily done
Census_OEMNameIdentifier done
Census_OEMModelIdentifier done
Census_ProcessorCoreCount done
Census_ProcessorManufacturerIdentifier done
Census_ProcessorModelIdentifier done
Census_ProcessorClass done
Census_PrimaryDiskTotalCapacity done
Census_PrimaryDiskTypeName done
Census_SystemVolumeTotalCapacity done
Census_HasOpticalDisk

In [18]:
df_tr.head().T

,0,1,2,3,4
ProductName,1,1,1,1,1
EngineVersion,2,5,2,2,2
AppVersion,1,5,1,1,1
AvSigVersion,80,2,183,17,14
IsBeta,1,1,1,1,1
RtpStateBitfield,1,1,1,1,1
IsSxsPassiveMode,1,1,1,1,1
DefaultBrowsersIdentifier,-1,-1,-1,-1,-1
AVProductStatesIdentifier,1,1,1,1,1
AVProductsInstalled,1,1,1,1,1


In [20]:
df_tr['Census_InternalBatteryNumberOfCharges'].unique()

[2.0, 4.0, 1.0, 29.0, 12.0, ..., 882.0, 785.0, 849.0, 825.0, 779.0]
Length: 899
Categories (899, float64): [0.0 < 1.0 < 2.0 < 3.0 ... 895.0 < 896.0 < 897.0 < 898.0]

# NEAT

In [19]:
df_tr.to_csv('tr_enc_big.csv',index=False)

In [21]:
df_te.to_csv('te_enc_big.csv',index=False)